In [32]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [33]:
# 内存优化配置
import joblib
from sklearn import config_context

# 设置全局并行作业数
config_context(print_changed_only=True)
joblib.parallel_backend('loky', n_jobs=4)  # 根据CPU核心数调整

In [34]:
# 加载数据
try:
    data = pd.read_csv('sales_data.csv')
except FileNotFoundError:
    print("数据集文件未找到，请检查文件名和路径。")

In [35]:
# 检查缺失值
print("缺失值情况：")
print(data.isnull().sum())

# 缺失值填充（这里简单用0填充，实际可根据情况调整）
data.fillna(0, inplace=True)

缺失值情况：
User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            166986
Product_Category_3            373299
Purchase                           0
dtype: int64


In [36]:
# 类别特征编码（将离散型分类变量转换为数值型哑变量）
# 定义需要独热编码的类别型特征列
categorical_columns = [
    'Gender',                   # 性别（二分类：男/女）
    'Age',                      # 年龄段（多分类，如18-25、26-35等）
    'Occupation',               # 职业类别（多分类编码）
    'City_Category',            # 城市分类（如A/B/C类城市）
    'Stay_In_Current_City_Years',# 在当前城市居住年限（有序分类）
    'Marital_Status'            # 婚姻状态（二分类：已婚/未婚）
]

# 初始化独热编码器对象
# sparse=True时返回稀疏矩阵，节省内存空间（sklearn>=1.2版本默认sparse_output=True）
encoder = OneHotEncoder(sparse_output=True)

# 同时进行编码器拟合（学习类别）和特征转换
# 输入必须是二维结构（DataFrame或二维数组），输出为稀疏矩阵格式
encoded_features = encoder.fit_transform(data[categorical_columns])

# 将稀疏矩阵转换为密集数组（numpy.ndarray）
# 原因：后续合并到DataFrame需要密集数据格式，且当前数据规模可承受内存消耗
# 注意：当类别数量极大时，此操作可能引发内存问题，此时应保持稀疏格式
encoded_array = encoded_features.toarray()

# 创建包含编码特征的DataFrame
# 使用编码器生成的列名（格式如：Gender_Female, Gender_Male）
# get_feature_names_out() 方法自动生成格式为"特征名_类别值"的列名
encoded_df = pd.DataFrame(
    encoded_array,
    columns=encoder.get_feature_names_out(categorical_columns)
)


In [37]:
# 数值特征标准化
numerical_columns = ['Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase']
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

In [38]:
# 合并编码后的类别特征和标准化后的数值特征
data = pd.concat([data.drop(categorical_columns, axis=1), encoded_df], axis=1)

# 划分训练集和测试集（以Purchase为预测目标，假设这是一个回归问题）
X = data.drop('Purchase', axis=1)
y = data['Purchase']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [39]:
# 注释：
# 导入必要的库，pandas用于数据处理，StandardScaler用于数值特征标准化，OneHotEncoder用于类别特征编码，train_test_split用于划分数据集。
# 尝试读取数据集，若文件不存在则提示错误。
# 检查数据集中的缺失值，并使用 0 填充缺失值（实际应用中可能需要更合理的填充策略）。
# 对类别特征进行独热编码，将编码后的特征转换为DataFrame。
# 对数值特征进行标准化处理。
# 将编码后的类别特征和标准化后的数值特征合并。
# 划分训练集和测试集，这里假设Purchase是预测目标，将数据集按 70% 训练集、30% 测试集的比例划分，并设置随机种子以确保结果可重复性。

In [40]:
# 分类 / 回归与聚类分析

In [41]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

# 加载数据
try:
    data = pd.read_csv('sales_data.csv')
except FileNotFoundError:
    print("数据集文件未找到，请检查文件名和路径。")

# 检查缺失值
print("缺失值情况：")
print(data.isnull().sum())

# 缺失值填充（这里简单用0填充，实际可根据情况调整）
data.fillna(0, inplace = True)

# 对Product_ID进行编码
le = LabelEncoder()
data['Product_ID'] = le.fit_transform(data['Product_ID'])

# 类别特征编码
categorical_columns = ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status']
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(data[categorical_columns])
encoded_df = pd.DataFrame(encoded_features.toarray(), columns = encoder.get_feature_names_out(categorical_columns))

# 数值特征标准化
numerical_columns = ['Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase']
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# 合并编码后的类别特征和标准化后的数值特征
data = pd.concat([data.drop(categorical_columns, axis = 1), encoded_df], axis = 1)

# 划分训练集和测试集（以Purchase为预测目标，假设这是一个回归问题）
X = data.drop('Purchase', axis = 1)
y = data['Purchase']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

缺失值情况：
User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            166986
Product_Category_3            373299
Purchase                           0
dtype: int64


In [42]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, silhouette_score
import numpy as np

In [43]:
# 回归任务：决策树回归
regressor = DecisionTreeRegressor(max_depth=5)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"决策树回归RMSE: {rmse:.2f}")


# # 聚类任务：K均值聚类
# kmeans = KMeans(n_clusters=3)
# clusters = kmeans.fit_predict(X)
# silhouette_avg = silhouette_score(X, clusters)
# print(f"K均值聚类轮廓系数: {silhouette_avg:.2f}")

决策树回归RMSE: 0.70


In [44]:
# 注释：
# 导入DecisionTreeRegressor用于回归任务，KMeans用于聚类任务，mean_squared_error用于计算回归的均方误差，silhouette_score用于评估聚类效果，np用于数值计算。
# 初始化决策树回归器，设置最大深度为 5，然后在训练集上进行训练，并在测试集上进行预测，计算并打印均方根误差（RMSE）。
# 初始化 K 均值聚类器，设置聚类数为 3，对整个数据集进行聚类，并计算和打印轮廓系数，以评估聚类效果。

In [45]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Adaboost回归（基于决策树）
ada = AdaBoostRegressor(estimator = DecisionTreeRegressor(max_depth = 2), n_estimators = 50)
ada.fit(X_train, y_train)
ada_rmse = np.sqrt(mean_squared_error(y_test, ada.predict(X_test)))
print(f"Adaboost回归RMSE: {ada_rmse:.2f}")

Adaboost回归RMSE: 0.79


In [46]:
from sklearn.ensemble import RandomForestRegressor

# 随机森林回归
rf = RandomForestRegressor(n_estimators=100, max_depth=10)
rf.fit(X_train, y_train)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf.predict(X_test)))
print(f"随机森林回归RMSE: {rf_rmse:.2f}")

随机森林回归RMSE: 0.56


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 定义基础估计器
base_estimator = DecisionTreeRegressor()

# 超参数调优 - Adaboost
param_grid_ada = {
    'n_estimators': [20, 50, 100],
    'estimator__max_depth': [1, 2, 3]
}
grid_search_ada = GridSearchCV(AdaBoostRegressor(estimator = base_estimator), param_grid_ada, cv = 5)
grid_search_ada.fit(X_train, y_train)
best_ada = grid_search_ada.best_estimator_
best_ada_rmse = np.sqrt(mean_squared_error(y_test, best_ada.predict(X_test)))
print(f"调优后Adaboost回归RMSE: {best_ada_rmse:.2f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# === 数据预处理 ===
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 处理缺失值（Product_Category_2/3存在空值）
data = pd.read_csv('purchase_data.csv')
data.fillna({'Product_Category_2':0, 'Product_Category_3':0}, inplace=True)

# 特征工程
# 年龄分段转换为数值（0-17 -> 0, 18-25 -> 1,...）
age_mapping = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
data['Age'] = data['Age'].map(age_mapping)

# 编码分类特征
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])
data['City_Category'] = le.fit_transform(data['City_Category'])

# === 类别不平衡处理 ===
from sklearn.utils import class_weight
classes = data['Purchase'].unique()
weights = class_weight.compute_class_weight('balanced', classes=classes, y=data['Purchase'])
class_weights = dict(zip(classes, weights))

# === 参数调优 ===
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'class_weight': [None, 'balanced', class_weights]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# === 特征重要性分析 ===
import matplotlib.pyplot as plt

rf_best = grid_search.best_estimator_
features = X_train.columns
importances = rf_best.feature_importances_

# === 特征重要性分析 ===（修正版本）
plt.figure(figsize=(10,6))
# 确保使用正确的特征名称（X.columns）
sorted_idx = importances.argsort()
plt.barh(X.columns[sorted_idx], importances[sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')

# === 内存优化（大规模数据示例）===
# 分块读取数据示例
chunk_size = 10**5
data_chunks = pd.read_csv('big_data.csv', chunksize=chunk_size)

# === 结果保存 ===（修正版本）
results = pd.DataFrame({
    'Model': ['AdaBoost', 'RandomForest', 'Tuned_AdaBoost'],
    'RMSE': [ada_rmse, rf_rmse, best_ada_rmse],  # 使用已计算的RMSE变量
    'Best_Params': [
        '',
        str(grid_search_ada.best_params_),  # 使用实际存在的调参对象
        str(grid_search.best_params_)
    ]
})
results.to_csv('model_evaluation.csv', index=False)

In [ ]:
# === 结果保存 ===（修正版本）
results = pd.DataFrame({
    'Model': ['AdaBoost', 'RandomForest', 'Tuned_AdaBoost'],
    'RMSE': [ada_rmse, rf_rmse, best_ada_rmse],  # 使用已计算的RMSE变量
    'Best_Params': [
        '',
        str(grid_search_ada.best_params_),  # 使用实际存在的调参对象
        str(grid_search.best_params_)
    ]
})
results.to_csv('model_evaluation.csv', index=False)